In [1]:
from bs4 import BeautifulSoup as bs
from Champions import ChampionWikia

In [2]:
CHAMPION_NAME = 'Thresh'

In [3]:
CHAMPION = ChampionWikia(CHAMPION_NAME)
CHAMPION.get()

200

In [4]:
html = bs(CHAMPION.html, 'html.parser')

In [5]:
for key, value in CHAMPION.base_stats.items():
    if html.find('span', id=value) is not None:
        print(value, html.find('span', id=value).string)
    else:
        print(value)

Health_Thresh 561
HealthRegen_Thresh 7
ResourceBar_Thresh 274
ResourceRegen_Thresh 6
Armor_Thresh 28
AttackDamage_Thresh 56
MagicResist_Thresh 30
MovementSpeed_Thresh 335
AttackRange_Thresh 450


In [6]:
for key, value in CHAMPION.base_stats_lvl.items():
    if html.find('span', id=value) is not None:
        print(value, float(html.find('span', id=value).string))
    else:
        print(value)

Health_Thresh_lvl 93.0
HealthRegen_Thresh_lvl 0.55
ResourceBar_Thresh_lvl 44.0
ResourceRegen_Thresh_lvl 0.8
Armor_Thresh_lvl 0.0
AttackDamage_Thresh_lvl 2.2
MagicResist_Thresh_lvl 0.5
MovementSpeed_Thresh_lvl
AttackRange_Thresh_lvl
